# Flask RestAPI demo

The main purpose of this demo is to show how to deploy a machine learning app using flask. <br>
For a more rigorious demo on building machine learning model, please refer to other notebooks.

In [1]:
import pickle
import requests
import pandas as pd
import json
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [2]:
# load data
data, target = load_iris(return_X_y=True)

In [3]:
# spliting data to train and test set
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [4]:
# building pipeline
preprocessor = StandardScaler()
model = LogisticRegression(C=3, multi_class='ovr')
pipeline = Pipeline([('standard_scaler', preprocessor), ('classifier', model)])
cross_validate(pipeline, X_train, y_train, cv=3)

{'fit_time': array([0.01300049, 0.01300073, 0.01100063]),
 'score_time': array([0.00400043, 0.        , 0.00099993]),
 'test_score': array([0.94736842, 0.86486486, 0.97297297])}

In [5]:
pipeline.fit(X_train, y_train)
print('The accuracy score for train set is {}'.format(accuracy_score(y_train, pipeline.predict(X_train))))
print('The accuracy score for test set is {}'.format(accuracy_score(y_test, pipeline.predict(X_test))))

The accuracy score for train set is 0.9375
The accuracy score for test set is 0.9473684210526315


In [6]:
with open('model.pkl', 'wb') as io:
    pickle.dump(pipeline, io)

The following script is the main code of the app, which accept an Http post request and return the prediction result.

In [7]:
%%writefile flask_app.py
import pickle
import numpy as np
from flask import Flask, request, jsonify

app = Flask(__name__)
with open('model.pkl', 'rb') as io:
    model = pickle.load(io)
    
@app.route('/predict', methods=['POST', 'GET'])
def predict():
    if request.method == 'GET':
        data = request.args
        data = [data.get(f'arg{index}') for index in range(1, 5)]
    else :
        data = request.get_json()
        data = [data.get(f'arg{index}') for index in range(1, 5)]
        return str(data)
        
    try :
        data = [float(record) for record in data if record]
        X_data = np.array(data).reshape(-1, 4)
        y_pred = model.predict(X_data)[0]
        y_prob = np.max(model.predict_proba(X_data))
        if request.method == 'GET':
            return f'It is in group {y_pred} with probability {y_prob:.2f}'
        else:
            result = {'group' : f'{y_pred}', 'probability': f'{y_prob:.2f}'}
            return jsonify(result)
    except ValueError: 
        return 'Please input 4 data with names arg1, arg2, arg3, arg4'
    

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')


Overwriting flask_app.py


Execute the code flask_app.py in the terminal. <br>
For example, in the windows environment, execute <br>
set FLASK_APP=flask_app.py <br>
flask run

### Get request

In [8]:
requests.get('http://127.0.0.1:5000/predict?arg1=6.8&arg2=2.8&arg3=4.8&arg4=1.4').content

b'It is in group 1 with probability 0.78'

In [9]:
requests.get('http://127.0.0.1:5000/predict?arg5=6.8&arg2=2.8&arg3=4.8&arg4=1.4').content

b'Please input 4 data with names arg1, arg2, arg3, arg4'

### Post Request

In [10]:
data = json.dumps({'arg1': '6.8',  'arg2': '2.8' , 'arg3': '4.8', 'arg4':'1.4'})
header = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
requests.post('http://127.0.0.1:5000/predict', data=data, headers=header).content

b'{"group":"1","probability":"0.78"}\n'

In [11]:
data = json.dumps({'arg5': '6.8',  'arg2': '2.8' , 'arg3': '4.8', 'arg4':'1.4'})
header = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
requests.get('http://127.0.0.1:5000/predict').content

b'Please input 4 data with names arg1, arg2, arg3, arg4'